# Analise de dados ENEM 2017

# 0.0 Imports

In [1]:
import inflection
import sqlite3
import zipfile
import pandas                   as pd
import numpy                    as np
import seaborn                  as sns
import plotly.express           as px
from sqlalchemy                 import create_engine
from matplotlib                 import gridspec
from matplotlib                 import pyplot as plt

from geopy.geocoders            import Nominatim
from geopy.extra.rate_limiter   import RateLimiter

## Funções

In [2]:
# Função para formatar notação científica
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Tamanho dos Graficos
sns.set(style = "darkgrid")
sns.set(rc={"figure.figsize":(15, 10)})
#pd.options.display.float_format = "{:,.2f}".format

def show_dimensions(data):
    print(f'Numbers of rows: { df.shape[0] }\n')
    print(f'Numbers of columns: { df.shape[1] }\n')
    return None

def show_dtypes(data):
    print(data.info())
    return None

# Obter coordenadas geográficas
def geo_data(data):
    locator = Nominatim(user_agent="mygeocoder")
    geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

    df_cidades['geolocalizacao'] = df_cidades['Município'].apply(geocode)
    ## Filtrando lat e long da coluna
    df_cidades['lat']= df_cidades['geolocalizacao'].apply(lambda loc: loc.latitude if loc else None)
    df_cidades['long']= df_cidades['geolocalizacao'].apply(lambda loc: loc.longitude if loc else None)
       
    return data

def clean_code(data):
    """Esta Função Limpa o DataFrame
    tipo de limpeza:
    remove os dados NA
    remove as notas zero """
    df = df.dropna()
    
    return data
    

##  Extração  dos dados

In [40]:
#MIcrodados Enem
colunas = ['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA','TP_ESCOLA', 
           'NO_MUNICIPIO_PROVA', 'SG_UF_PROVA','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT','NU_NOTA_REDACAO','Q006']

with zipfile.ZipFile('data/microdados_enem_2017.zip') as z:
    with z.open('DADOS/MICRODADOS_ENEM_2017.csv') as f:
        # Criar uma lista com estas colunas que serão usadas
        df = pd.read_csv(f,sep=';', encoding='ISO-8859-1', usecols=colunas, nrows=25000)
        
# carregar o Dataset cidades
#colunas = ['Município','IDHMunicipal_2010', 'População_estimada_pessoas_2019']

#df_cidades = pd.read_csv('data/cidades.csv', usecols=colunas)


In [41]:
colunas = ['Município','IDHMunicipal_2010', 'População_estimada_pessoas_2019']

df_cidades = pd.read_csv('data/cidades.csv', usecols=colunas)


In [42]:
df_cidades.head()

,Município,População_estimada_pessoas_2019,IDHMunicipal_2010
0,Adamantina,35068,"0,79"
1,Adolfo,3562,"0,73"
2,Aguaí,36305,"0,715"
3,Águas da Prata,8180,"0,781"
4,Águas de Lindóia,18705,"0,745"


In [43]:
cols = ['Município','IDHMunicipal_2010', 'lat', 'long']
df_coord= pd.read_csv('data/cidade_geolocalizacao.csv', usecols=cols)

In [44]:
df_coord['populacao'] = df_cidades['População_estimada_pessoas_2019']
df_coord.head()


,Município,IDHMunicipal_2010,lat,long,populacao
0,Adamantina,"0,79",-21.6867,-51.0763,35068
1,Adolfo,"0,73",-21.2357,-49.6442,3562
2,Aguaí,"0,715",-22.0592,-46.9794,36305
3,Águas da Prata,"0,781",-21.9464,-46.7192,8180
4,Águas de Lindóia,"0,745",-22.4719,-46.6291,18705


In [45]:
df_cidades = df_coord.copy()

In [46]:
df_cidades.columns =[['NO_MUNICIPIO_PROVA', 'idh', 'lat', 'long', 'populacao']]

In [47]:
#Renomear colunas para concatenar os datasets
df_cidades.columns=['NO_MUNICIPIO_PROVA', 'idh', 'lat', 'long', 'populacao',]

df = pd.merge(df, df_cidades, how='left', on='NO_MUNICIPIO_PROVA')

In [48]:
show_dimensions(df)

show_dtypes(df)

Numbers of rows: 27669

Numbers of columns: 19

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27669 entries, 0 to 27668
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NU_INSCRICAO        27669 non-null  int64  
 1   NU_ANO              27669 non-null  int64  
 2   TP_FAIXA_ETARIA     27669 non-null  int64  
 3   TP_SEXO             27669 non-null  object 
 4   TP_ESTADO_CIVIL     27669 non-null  int64  
 5   TP_COR_RACA         27669 non-null  int64  
 6   TP_ESCOLA           27669 non-null  int64  
 7   NO_MUNICIPIO_PROVA  27669 non-null  object 
 8   SG_UF_PROVA         27669 non-null  object 
 9   NU_NOTA_CN          19958 non-null  float64
 10  NU_NOTA_CH          20877 non-null  float64
 11  NU_NOTA_LC          20877 non-null  float64
 12  NU_NOTA_MT          19958 non-null  float64
 13  NU_NOTA_REDACAO     20877 non-null  float64
 14  Q006                27669 non-null  object 
 15  idh  

##  Transformação

In [49]:
#Renomear as colunas do dataset microdados
cols_old = ['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ESCOLA',
       'NO_MUNICIPIO_PROVA', 'SG_UF_PROVA', 'NU_NOTA_CN', 'NU_NOTA_CH',
       'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q006', 'idh', 'lat',
       'long', 'populacao']


#Função para renomear colunas
snakecase = lambda x: inflection.underscore(x)
cols_new = list( map( snakecase, cols_old) )
#Rename
df.columns = cols_new

In [50]:
df.columns = ['inscricao', 'ano', 'faixa_etaria', 'sexo',
       'estado_civil', 'cor_raca', 'tp_escola',
       'municipio', 'estado', 'ciencias_natureza', 'ciencias_humanas',
       'linguagens_codigos', 'matematica', 'redacao', 'renda_familiar', 'idh', 'lat',
       'long', 'populacao']

In [51]:
df.isna().sum()

inscricao                0
ano                      0
faixa_etaria             0
sexo                     0
estado_civil             0
cor_raca                 0
tp_escola                0
municipio                0
estado                   0
ciencias_natureza     7711
ciencias_humanas      6792
linguagens_codigos    6792
matematica            7711
redacao               6792
renda_familiar           0
idh                      0
lat                      0
long                     0
populacao                0
dtype: int64

In [52]:
#Exluindo os valores nulos
df = df.dropna()

In [53]:
show_dimensions(df)

Numbers of rows: 19941

Numbers of columns: 19



In [54]:
# Cria nota média
df['nota_media'] = (df['ciencias_natureza']+
                    df['ciencias_humanas'] +
                    df['linguagens_codigos']+ 
                    df['matematica'] +
                    df['redacao']) / 5

#eliminando os estudantes que ficaram com nota 0 por qualquer outro motivo (e.g. eliminados)
df = df[df['nota_media'] != 0]

In [55]:
regiao = {
    "Norte":['AM', 'PA', 'RR', 'AC','RO','AP', 'TO'],
    "Sul":['PR', 'SC', 'RS'],
    "Centro_Oeste":['MT', 'MS', 'GO', 'DF'],
    "Nordeste":['BA', 'CE','MA', 'PB', 'PE', 'RN', 'SE', 'AL', 'PI'],
    "Sudeste":['SP', 'RJ', 'MG','ES']}

for r in regiao:
    df.loc[ df['estado'].isin(regiao[r]), 'regiao'] = r

## Insert Data

In [56]:
df.shape

(19941, 21)

- Criar um banco de dados com uma amostra de 826668 linhas de dados

In [57]:
query_create_table_microdados_enem =  """
    CREATE TABLE microdados_enem (
      'inscricao'           INTEGER,
      'ano'                 INTEGER,
      'faixa_etaria'        INTEGER,
      'sexo'                TEXT,
      'estado_civil'        TEXT,
      'cor_raca'            INTEGER,
      'tp_escola'           INTEGER,
      'tp_ensino'           REAL,
      'municipio'           TEXT,
      'estado'              TEXT,
      'ciencias_natureza'   REAL,
      'ciencias_humanas'    REAL,
      'linguagens_codigos'  REAL,
      'matematica'          REAL,
      'redacao'             REAL,
      'renda_familiar'      TEXT, 
      'idh'                 REAL,
      'lat'                 REAL,
      'long'                REAL,
      'populacao'           REAL,
      'nota_media'          REAL,
      'regiao'              TEXT    
    )
"""

In [60]:
query_drop = """ 
DROP TABLE microdados_enem
"""

In [62]:
#Create table
conn = sqlite3.connect('db_enem.sqlite')
#conn.execute(query_drop)
conn.execute(query_create_table_microdados_enem)
conn.commit()
conn.close()

In [63]:
conn = create_engine('sqlite:///db_enem.sqlite', echo=False)

#insert data
df.to_sql('microdados_enem', con=conn, if_exists='append', index=False)

19941

In [64]:
#Consultando o banco de dados
query ="""
    SELECT *  FROM microdados_enem
"""
pd.read_sql_query(query, conn)
df.shape

(19941, 21)

In [65]:
data= df.sample(10000).copy()

In [66]:
data.to_csv('data/enem.csv', index=False)